In [4]:
from __future__ import print_function

import numpy as np
import matplotlib.pyplot as plt 
from scipy.stats import norm 
import tensorflow as tf 

from tensorflow.keras.layers import Input, Dense, Lambda, Reshape
from tensorflow.keras.models import Model 
from tensorflow.keras import backend as K
from tensorflow.keras import metrics
from tensorflow.keras.datasets  import mnist


In [7]:
# Defining the key parameters
batch_size = 100
original_dim = 784 # 28x28
latent_dim = 2
intermediate_dim = 256
epochs = 50
epsilon_std = 1.0

In [8]:
# Define sampling elper function
def sampling(args: tuple):
    # We grab the variables from the tuple
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim), mean=0., stddev=epsilon_std)

    return z_mean + K.exp(z_log_var / 2) * epsilon
                        

## Defining the encoder

In [9]:
# input to our encoder
x = Input(shape=(original_dim,), name='input')

# intermediate layer
h = Dense(intermediate_dim, activation='relu', name='encoding')(x)

# defining the mean of the laten space
z_mean = Dense(latent_dim, name='mean')(h)

# defining the log variance of the laten space
z_log_var = Dense(latent_dim, name='log_variance')(h)

# note that 'output_shape' isn't necessary with the TensorFlow backend
z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])

# defining the encoder as a keras model
encoder = Model(x, [z_mean, z_log_var, z], name='encoder')

# print out summary of what we just did
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 784)]        0                                            
__________________________________________________________________________________________________
encoding (Dense)                (None, 256)          200960      input[0][0]                      
__________________________________________________________________________________________________
mean (Dense)                    (None, 2)            514         encoding[0][0]                   
__________________________________________________________________________________________________
log_variance (Dense)            (None, 2)            514         encoding[0][0]                   
____________________________________________________________________________________________

## Define the decoder


In [10]:
# Input to the decoder
input_decoder = Input(shape=(latent_dim,), name='input_decoder')

# taking the latent space to intermediate dimension
decoder_h = Dense(intermediate_dim, activation='relu', name='decoder_h')(input_decoder)

# getting the mean from the original dimension
x_decoded = Dense(original_dim, activation='relu', name='flat_decoded')(decoder_h)

# define the decoder as a keras model
decoder = Model(input_decoder, x_decoded, name='decoder')

# print out summary of model
decoder.summary()

Model: "decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_decoder (InputLayer)   [(None, 2)]               0         
_________________________________________________________________
decoder_h (Dense)            (None, 256)               768       
_________________________________________________________________
flat_decoded (Dense)         (None, 784)               201488    
Total params: 202,256
Trainable params: 202,256
Non-trainable params: 0
_________________________________________________________________


## Define the Variational Autoencoder (VAE)

In [11]:
# grab the output, recall, that we need to grab the 3rd element our sampling z
output_combined = decoder(encoder(x)[2])

# link the input and the overall output
vae = Model(x, output_combined)

# print out summary of model
vae.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 784)]             0         
_________________________________________________________________
encoder (Model)              [(None, 2), (None, 2), (N 201988    
_________________________________________________________________
decoder (Model)              (None, 784)               202256    
Total params: 404,244
Trainable params: 404,244
Non-trainable params: 0
_________________________________________________________________


## Define the losses and run the model!

In [12]:
def vae_loss(x: tf.Tensor, x_decoded_mean: tf.Tensor, z_log_var=z_log_var, z_mean=z_mean, original_dim=original_dim):
    xent_loss = original_dim * metrics.binary_crossentropy(x, x_decoded_mean)
    kl_loss = -0.5 * K.sum(
        1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
    
    vae_loss = K.mean(xent_loss + kl_loss)

    return vae_loss 


vae.compile(optimizer='rmsprop', loss=vae_loss)
vae.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 784)]             0         
_________________________________________________________________
encoder (Model)              [(None, 2), (None, 2), (N 201988    
_________________________________________________________________
decoder (Model)              (None, 784)               202256    
Total params: 404,244
Trainable params: 404,244
Non-trainable params: 0
_________________________________________________________________


# Work with MNIST Data

In [13]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_test  = x_test.astype('float32') / 255. 
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test  = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

In [17]:
vae.fit(x_train, y_train,
        shuffle=True,
        epochs=epochs,
        batch_size=batch_size)

Train on 60000 samples
Epoch 1/50
  100/60000 [..............................] - ETA: 4s

_SymbolicException: Inputs to eager execution function cannot be Keras symbolic tensors, but found [<tf.Tensor 'log_variance_1/Identity:0' shape=(None, 2) dtype=float32>, <tf.Tensor 'mean_1/Identity:0' shape=(None, 2) dtype=float32>]